<a href="https://colab.research.google.com/github/NunezKant/COVID19_MEX_MASTER/blob/master/Bayesian_MCMC_fitter_State_Agnostic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install pymc3==3.8

%matplotlib inline
import numpy as np
from IPython.display import display, Markdown
import matplotlib.pyplot as plt
import matplotlib
import pandas as pd
import seaborn as sns

import pymc3 as pm
from pymc3.ode import DifferentialEquation
import numpy as np
import matplotlib.pyplot as plt
from scipy.integrate import odeint
import arviz as az
import theano

plt.style.use('seaborn-darkgrid')

Definimos la ecuación diferencial SEIR - utilzamos un modelo simplificado para fittear y obtener la Ro

In [0]:
def SEIR(y, t, p):
    """
    y[0] = S
    y[1] = E
    y[2] = I
    y[3] = R

    
    p[0] = Alpha
    p[1] = Beta
    p[2] = Gamma


    St = S[-1] - (beta*S[-1]*I[-1])*dt
    Et = E[-1] + (beta*S[-1]*I[-1] - alpha*E[-1])*dt
    It = I[-1] + rho*(alpha*E[-1] - gamma*I[-1])*dt
    Rt = gamma*I[-1]
    """



    ds = -p[1]*y[0]*y[2]
    de = p[1]*y[0]*y[2] - p[0]*y[1]
    di = p[0]*y[1] - p[2]*y[2]
    dr = p[2]*y[2]
    return [ds, de, di, dr]

FUNCIONES DE UTILIDAD

In [0]:
def TimeSeriesLoader(url):

  """
  Este Loader se puede cambiar según la fuente, en ese caso, tambien habría que cambiar un poco la función
  GetObs_and_times(estado,Pop)
  """
  df = pd.read_csv(url)

  df['Reporte'] = pd.to_datetime(df['Reporte'])
  df = (df.drop(["Procedencia", "Llegada","Inicio","Caso","Sexo","Edad","Confirmación"], axis=1)
          .dropna()
          .rename(columns = {"Reporte":"Dia"})
          .sort_values(by = ["Dia"])
          .reset_index(drop = True)
        )
  df["Casos_Acumulados"] = np.nan
  Estados = pd.DataFrame({
    "Estado":[],
    "Dia":[],
    "Casos_Acumulados":[]
  })

  for estado in df.Estado.unique():
      df.loc[(df.Estado == estado),"Casos_Acumulados"] =  np.arange(1,len(df.loc[(df.Estado == estado),"Casos_Acumulados"])+1,1)
      tmp = df.loc[df.Estado == estado].drop_duplicates("Dia", keep = 'last').reset_index(drop = True)
      Estados = pd.concat([Estados,tmp])
  return Estados

In [0]:
def GetObs_and_times(estado,Pop,r,d):
  from scipy.interpolate import InterpolatedUnivariateSpline
  """
  Esta funcion genera las condiciones inciales para PyMC3 a partir de las observaciones de 
  Infectados en un estado y dada la población del estado (se debe ingresar el número)
  como los registros de recuperados en mexico son nulos se asumen 0.
  """
  yobs = Estados.loc[Estados.Estado == estado,"Casos_Acumulados"].values
  days = Estados.loc[Estados.Estado == estado,"Dia"].dt.day.values
  days = (days - days[0])
  times = np.arange(0,days[-1]+1,1)
  
  interp = InterpolatedUnivariateSpline(days,yobs,k=2)
  obs_interp = interp(times)


  def SEIR_Day(obs,r=0, d=0):
    I = obs / Pop
    E = (obs*4) / Pop # Ojo aca con el 4
    D = d
    R = r / Pop
    S = 1 - (E-I-R)
    return [S, E, I, R]

  yobs_arr = np.array([SEIR_Day(obs,r,d) for obs in obs_interp])

## Crear un arreglo de N x 4 para enviar como estados al sistema


  return obs_interp, yobs_arr, times

Condiciones iniciales para la ODE, verificamos salida de ODEINT

In [0]:
np.random.seed(666)

# JALISCO (2020-03-14)
Population = 8000000
I_o = 2 / Population  # Tenemos 32 casos
E_o = (2*4)/ Population # Asumimos 4 expuestos por caso
S_o = (1) - (E_o+I_o) # El resto somos suceptibles
R_o = 0 # NO hay ningun recuperado

dias_evaluacion = 120
dt=1
periodo_evaluacion = np.arange(0,dias_evaluacion+dt,dt)

R_o = 3.5 #Escenario Base
alpha = 0.2
gamma = 0.5
beta = R_o * gamma

y0 = np.array([S_o,E_o,I_o,R_o])

args = ((alpha, beta, gamma,),)
 
y = odeint(SEIR,t=periodo_evaluacion,y0=y0, args=args, rtol=1e-08)

In [38]:
#@title Salida de ODEINT { display-mode: "form" }
import plotly.express as px
E_a = y[:,1]*Population
I_a = y[:,2]*Population
Clase = np.array(["Expuestos"]*y[:,1].shape[0] + ["Infectados"]*y[:,1].shape[0])
Dias = np.concatenate([periodo_evaluacion,periodo_evaluacion])
SEIR_df = pd.DataFrame({
    "Casos": np.concatenate([E_a,I_a]),
    "Clase": Clase,
    "Dias" : Dias
})

fig = px.line(SEIR_df, x="Dias", y="Casos", color='Clase',color_discrete_sequence=["green", "red", "goldenrod", "blue"], template = "ggplot2")

fig.update_layout(
    title=f"Predicción del modelo SEIR de la evolución de COVID-19 en Jalisco, Escenario: BASE Ro = 3.5",
    
    xaxis_title="Días",
    yaxis_title="Casos Totales",
    )

for trace in fig.data:
    trace.name = trace.name.split('=')[1]

fig.show()

Por lo tanto está bien definido el sistema de ecuaciones.

In [39]:
Estados = TimeSeriesLoader("https://gist.githubusercontent.com/said3427/18f39eab460766a17ea4802607dd5522/raw")
Estados.head()

,Estado,Dia,Casos_Acumulados
0,Ciudad De México,2020-02-27,1.0
1,Ciudad De México,2020-02-28,2.0
2,Ciudad De México,2020-03-07,3.0
3,Ciudad De México,2020-03-11,5.0
4,Ciudad De México,2020-03-13,11.0


In [0]:
y_interpol, obs,times = GetObs_and_times("Jalisco",8000000,0,0)

In [0]:
seir_model = DifferentialEquation(
    func=SEIR,
    times=times,
    n_states=4,
    n_theta=3,
)

In [0]:
sigma = 10 / Population # numero de error (prior) de casos por dia aquí se asumen 10

In [46]:
with pm.Model() as model:

    sigma = pm.HalfCauchy('sigma', sigma, shape=4) 
    #p_alpha = pm.Bound(pm.Normal, lower=0, upper=1)("alpha",.5, 0.3 )  
    p_gamma = pm.Bound(pm.Normal, lower=0, upper=1)("gamma",.5, 0.37 )  
    R0 = pm.Bound(pm.Normal, lower=1, upper=5)('R0', 3.5, 1)
    p_beta = pm.Deterministic('beta', gamma*R0)
    


    seir_curves = seir_model(y0=obs[0], theta=[alpha, p_beta, p_gamma])
    Y = pm.Normal('Y', mu=seir_curves, sd=sigma, observed=obs)
    
    prior = pm.sample_prior_predictive()
    trace = pm.sample(1000,tune=500, target_accept=0.95, cores=2)
    posterior_predictive = pm.sample_posterior_predictive(trace)

    data = az.from_pymc3(trace=trace, prior = prior, posterior_predictive = posterior_predictive)

Auto-assigning NUTS sampler...
Initializing NUTS using jitter+adapt_diag...
Multiprocess sampling (2 chains in 2 jobs)
NUTS: [R0, gamma, sigma]
Sampling 2 chains, 326 divergences:  45%|████▍     | 1344/3000 [1:21:19<1:40:12,  3.63s/draws]
Only one chain was sampled, this makes it impossible to run some convergence checks
100%|██████████| 251/251 [00:08<00:00, 30.93it/s]


IndexError: ignored

In [0]:
az.plot_posterior(data,round_to=2, credible_interval=0.95)

In [0]:
ppc_samples = posterior_predictive["Y"]
mean_ppc = ppc_samples.mean(axis=0)
CriL_ppc = np.percentile(ppc_samples,q=2.5,axis=0)
CriU_ppc = np.percentile(ppc_samples,q=97.5,axis=0)
plt.plot(times,y_interpol,'o', color='b', lw=1, label='Infected cases observed')
plt.plot(times,mean_ppc[:,3]*Population, color='g', lw=4, label=f'mean of $I(t)$ ppc')
plt.plot(times,CriL_ppc[:,3]*Population, '--',  color='g', lw=2, label=f'$I(t)$ credible intervals')
plt.plot(times,CriU_ppc[:,3]*Population, '--',  color='g', lw=2)
plt.legend()

In [0]:
plt.figure(figsize = (7,5))
plt.hist(prior["R0"],histtype="stepfilled",bins=30,alpha = .80, label = f"Prior of $R_0$",color = "#A60628", density = True)
plt.hist(trace["R0"],histtype="stepfilled",bins=30,alpha = .80, label = f"Predictive Prior of $R_0$",color = "#467821", density = True)
plt.legend()